In [4]:
import cv2
import os

In [5]:
#dataset 영상 이름 변환
folder_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + '\\dataset'
folder_list = os.listdir(folder_path)

for folder_name in folder_list :
    file_path = folder_path + "\\" + folder_name
    file_list = os.listdir(file_path)

    count = 1
    for file_name in file_list :
        old_name = file_path + "\\" + file_name
        new_name = file_path + "\\" + folder_name + str(count).zfill(3) + ".mp4"
        try:
            os.rename(old_name, new_name)
            print("success : " + file_name + " -> " + folder_name + str(count).zfill(3) + ".mp4")
        except:
            print("fail : " + file_name + " -> already exists.")

        count += 1

success : backward001.mp4 -> backward001.mp4
success : backward002.mp4 -> backward002.mp4
success : backward003.mp4 -> backward003.mp4
success : slide001.mp4 -> slide001.mp4
success : slide002.mp4 -> slide002.mp4
success : slide003.mp4 -> slide003.mp4
success : slide004.mp4 -> slide004.mp4
success : slide005.mp4 -> slide005.mp4
success : stand001.mp4 -> stand001.mp4
success : swing001.mp4 -> swing001.mp4
success : swing002.mp4 -> swing002.mp4
success : swing003.mp4 -> swing003.mp4
success : swing004.mp4 -> swing004.mp4
success : swing005.mp4 -> swing005.mp4


In [11]:
action = ['backward', 'lie', 'sit', 'slide', 'stand', 'swing', 'walk']


In [2]:
def get_frames(action, seq): #행동 / 그 안의 영상 번호   #action에도 번호 부여
    video_path = f'../dataset/{action}/{action}{str(seq).zfill(3)}.mp4'

    if not os.path.exists(video_path):
        raise IOError('No video:' + video_path)

    frames = []

    cap = cv2.VideoCapture(video_path)

    max_len = 100
    sample_interval=1
    cnt = 0
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    fps = cap.get(cv2.CAP_PROP_FPS)

    if cap.isOpened():
        while True:
            ret, img = cap.read()
            if not ret:
                break
            if cnt >= max_len:
                break
            if int(cap.get(sample_interval)) % fps == 0:
                img = cv2.normalize(src=img, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
            frames.append(img)
            cnt += 1
        while len(frames) < max_len:
            frames.append(0)

    return frames

In [5]:
frames = get_frames('slide', 1)
resized_frame = cv2.resize(frames[80], dsize =(480, 720), interpolation=cv2.INTER_LINEAR)
print(resized_frame.shape)
cv2.imshow('asdf', resized_frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

(720, 480, 3)


In [17]:
cap = cv2.VideoCapture('E:\\2023\\2023_1_1\\rne\dataset\slide\slide001.mp4')
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(frame_count)

149


In [45]:
from dev.getJoints import outputkeypoints

proto_file = 'E:\\2023\\2023_1_1\\rne\openpose\models\pose\coco\pose_deploy_linevec.prototxt'
weights_file = 'E:\\2023\\2023_1_1\\rne\openpose\models\pose\coco\pose_iter_440000.caffemodel'
i = cv2.imread("E:\im\\fullshot.jpg")
frame = outputkeypoints(i, proto_file, weights_file)

[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]


error: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'blobFromImage'
> Overload resolution failed:
>  - Can't parse 'size'. Sequence item with index 0 has a wrong type
>  - Can't parse 'size'. Sequence item with index 0 has a wrong type


In [39]:
print(cv2.__version__)

4.8.0


In [ ]:
def extract_multi_frame_features(X, Y, video_indices, window_size):
    ''' From image index and raw skeleton positions,
        Extract features of body velocity, joint velocity, and normalized joint positions.
    '''
    X_new = []
    Y_new = []

    N = len(video_indices)

    # Loop through all data
    for i, _ in enumerate(video_indices):

        # If a new video clip starts, reset the feature generator
        if i == 0 or video_indices[i] != video_indices[i-1]:
            fg = FeatureGenerator(window_size, is_adding_noise)

        # Get features
        success, features = fg.add_cur_skeleton(X[i, :])
        if success:  # True if (data length > 5) and (skeleton has enough joints)
            X_new.append(features)
            Y_new.append(Y[i])

        # Print
        if is_print and i % 1000 == 0:
            print(f"{i}/{N}", end=", ")

    if is_print:
        print("")
    X_new = np.array(X_new)
    Y_new = np.array(Y_new)
    return X_new, Y_new


In [ ]:
def get_joint(x, idx):
    px = x[2*idx]
    py = x[2*idx+1]
    return px, py